data processing


In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv('/gdrive/My Drive/CICIDS2017/dataset/cicids2017_merge.csv')

In [ ]:
# df = df [['Subflow Fwd Bytes', 
#          'Bwd Packet Length Std',
#          'Flow Duration', 'Fwd IAT Mean','Init_Win_bytes_forward',      
#     'Label']]
df = df [['Subflow Fwd Bytes',
    'Label']]

In [ ]:
len(df)

2830743

In [ ]:
df = df[0:100000]

In [ ]:
df

,Subflow Fwd Bytes,Label
0,12,BENIGN
1,6,BENIGN
2,6,BENIGN
3,6,BENIGN
4,12,BENIGN
...,...,...
99995,24,DDoS
99996,24,DDoS
99997,24,DDoS
99998,30,DDoS


set up wrong label rate

In [ ]:
change_rate = 0.1

set DDos to benign, benign to ddos

In [ ]:
wrong_label_index = np.random.randint(0, len(df), size=(int(len(df)*change_rate)))

In [ ]:
t = df.iloc[list(wrong_label_index),[1]] == 'DDoS'

In [ ]:
df.loc[23582]

Subflow Fwd Bytes        24
Label                BENIGN
Name: 23582, dtype: object

In [ ]:
if 80630 in t.index:
  print("yes")

yes


In [ ]:
t

,Label
23582,True
16307,False
55471,True
80531,True
4370,False
...,...
63029,True
29250,True
93092,True
42481,False


In [ ]:
DDoS_index = t[t['Label'] == 1].index.tolist()

In [ ]:
Benign_index = t[t['Label'] == 0].index.tolist()

In [ ]:
df.iloc[DDoS_index, [1]] = "BENIGN"
df.iloc[Benign_index, [1]] = "DDoS"

In [ ]:
df

,Subflow Fwd Bytes,Label
0,12,BENIGN
1,6,BENIGN
2,6,BENIGN
3,6,BENIGN
4,12,BENIGN
...,...,...
99995,24,DDoS
99996,24,DDoS
99997,24,BENIGN
99998,30,DDoS


In [ ]:
df['Label'].value_counts()

DDoS      59043
BENIGN    40957
Name: Label, dtype: int64

In [ ]:
df.columns[~(df != 0).any(axis=0)]

Index([], dtype='object')

In [ ]:
df 

,Subflow Fwd Bytes,Label
0,12,BENIGN
1,6,BENIGN
2,6,BENIGN
3,6,BENIGN
4,12,BENIGN
...,...,...
99995,24,DDoS
99996,24,DDoS
99997,24,BENIGN
99998,30,DDoS


remove all zero

In [ ]:
df['Label'].value_counts().index

Index(['DDoS', 'BENIGN'], dtype='object')

In [ ]:
df = df.loc[:, (df != 0).any(axis=0)]

renove nan and inf

In [ ]:
df.replace([-np.inf, np.inf], np.nan, inplace=True)
df.dropna(inplace=True)

select features

In [ ]:
df.columns.values

array(['Subflow Fwd Bytes', 'Label'], dtype=object)

In [ ]:
#df = df[['Bwd Packet Length Max', 'Fwd Packet Length Max', 'Bwd Packet Length Std','Bwd Packet Length Min',
 #      'Label']]

In [ ]:
df

,Subflow Fwd Bytes,Label
0,12,BENIGN
1,6,BENIGN
2,6,BENIGN
3,6,BENIGN
4,12,BENIGN
...,...,...
99995,24,DDoS
99996,24,DDoS
99997,24,BENIGN
99998,30,DDoS


split test and train

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
seed = 1

In [ ]:
train_set, test_set = train_test_split(df, test_size=0.3, random_state=seed)

split data and target

In [ ]:
train_X_orgin = train_set.drop('Label', axis=1)
train_y_orgin = train_set[['Label']]
test_X_orgin = test_set.drop('Label', axis=1)
test_y_orgin = test_set[['Label']]

In [ ]:
for i in range(0, len(train_X_orgin)):
  if train_X_orgin['Subflow Fwd Bytes'].index[i] == 23582:
    print(i)
    break

41525


In [ ]:
train_X_orgin.values[41525]

array([24])

In [ ]:
train_X_orgin

,Subflow Fwd Bytes
80630,24
62017,24
5005,70
56849,56
42434,6
...,...
50057,66
98047,56
5192,611
77708,56


In [ ]:
train_y_orgin

,Label
80630,BENIGN
62017,DDoS
5005,BENIGN
56849,DDoS
42434,BENIGN
...,...
50057,BENIGN
98047,DDoS
5192,BENIGN
77708,BENIGN


In [ ]:
train_index = train_X_orgin.index.tolist()
test_index = test_X_orgin.index.tolist()

In [ ]:
train_y_orgin.values

array([['BENIGN'],
       ['DDoS'],
       ['BENIGN'],
       ...,
       ['BENIGN'],
       ['BENIGN'],
       ['DDoS']], dtype=object)

normalization

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
Scaler = StandardScaler()

In [ ]:
train_X = Scaler.fit_transform(train_X_orgin)
test_X = Scaler.transform(test_X_orgin)

In [ ]:
train_X.shape

(70000, 1)

encode

In [ ]:
from sklearn.preprocessing import OneHotEncoder

In [ ]:
y_test_ada = test_y_orgin

model

In [ ]:
from xgboost import XGBClassifier

In [ ]:
model = XGBClassifier()
model.fit(train_X, train_y_orgin.values.ravel())

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [ ]:
train_y_orgin.shape

(70000, 1)

In [ ]:
import tensorflow as tf

In [ ]:
METRICS = [
      tf.keras.metrics.TruePositives(name='tp'),
      tf.keras.metrics.FalsePositives(name='fp'),
      tf.keras.metrics.TrueNegatives(name='tn'),
      tf.keras.metrics.FalseNegatives(name='fn'), 
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall'),
      #tf.keras.metrics.AUC(name='auc'),
]

metrics=METRICS

In [ ]:
y_pred=model.predict(test_X)

In [ ]:
y_pred

array(['DDoS', 'BENIGN', 'BENIGN', ..., 'BENIGN', 'DDoS', 'BENIGN'],
      dtype=object)

In [ ]:
def display_metrics(y_test, y_pred, label_names):
  print('\nAccuracy: {:.2f}\n'.format(accuracy_score(y_test, y_pred)))

  print('Micro Precision: {:.2f}'.format(precision_score(y_test, y_pred, average='micro')))
  print('Micro Recall: {:.2f}'.format(recall_score(y_test, y_pred, average='micro')))
  print('Micro F1-score: {:.2f}\n'.format(f1_score(y_test, y_pred, average='micro')))

  print('Macro Precision: {:.2f}'.format(precision_score(y_test, y_pred, average='macro')))
  print('Macro Recall: {:.2f}'.format(recall_score(y_test, y_pred, average='macro')))
  print('Macro F1-score: {:.2f}\n'.format(f1_score(y_test, y_pred, average='macro')))

  print('Weighted Precision: {:.2f}'.format(precision_score(y_test, y_pred, average='weighted')))
  print('Weighted Recall: {:.2f}'.format(recall_score(y_test, y_pred, average='weighted')))
  print('Weighted F1-score: {:.2f}'.format(f1_score(y_test, y_pred, average='weighted')))

  print('\nClassification Report\n')
  print(classification_report(y_test, y_pred, target_names=label_names))

In [ ]:
def make_value2index(attacks):
    #make dictionary
    attacks = sorted(attacks)
    d = {}
    counter=0
    for attack in attacks:
        d[attack] = counter
        counter+=1
    return d
    

In [ ]:
def encode_label(Y_str):
    labels_d = make_value2index(np.unique(Y_str))
    Y = [labels_d[y_str] for y_str  in Y_str]
    Y = np.array(Y)
    return np.array(Y)

In [ ]:
labels_d = make_value2index(y_test_ada['Label'])

In [ ]:
#importing confusion matrix
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

from sklearn import metrics
from sklearn.metrics import accuracy_score

#importing accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import mean_squared_error,mean_absolute_error

In [ ]:
display_metrics(test_y_orgin, y_pred, labels_d)


Accuracy: 0.90

Micro Precision: 0.90
Micro Recall: 0.90
Micro F1-score: 0.90

Macro Precision: 0.90
Macro Recall: 0.89
Macro F1-score: 0.89

Weighted Precision: 0.90
Weighted Recall: 0.90
Weighted F1-score: 0.90

Classification Report

              precision    recall  f1-score   support

      BENIGN       0.90      0.85      0.87     12286
        DDoS       0.90      0.94      0.92     17714

    accuracy                           0.90     30000
   macro avg       0.90      0.89      0.89     30000
weighted avg       0.90      0.90      0.90     30000



LIME 

In [ ]:
!pip install shap

In [ ]:
import shap

In [ ]:
explainer = shap.Explainer(model)
shap_values = explainer(train_X)

In [ ]:
shap_values.values[41525]

array([1.7050837], dtype=float32)

In [ ]:
len(train_X)

70000

In [ ]:
shap_values

.values =
array([[ 1.7050837],
       [ 1.7050837],
       [-2.693884 ],
       ...,
       [-2.9287174],
       [ 1.562161 ],
       [ 1.7287644]], dtype=float32)

.base_values =
array([0.46815172, 0.46815172, 0.46815172, ..., 0.46815172, 0.46815172,
       0.46815172], dtype=float32)

.data =
array([[-0.22723126],
       [-0.22723126],
       [-0.21248181],
       ...,
       [-0.03901541],
       [-0.21697077],
       [-0.22658998]])

In [ ]:
print(model.predict_proba(-0.23300279))

[[0.90639883 0.09360119]]


In [ ]:
print(model)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)


In [ ]:
import math

def sigmoid(x):
  return 1 / (1 + math.exp(-x))

In [ ]:
sigmoid(-5.366)

0.004651051752853609

In [ ]:
df2 = pd.DataFrame(shap_values.values, columns=['shap_values'], index=train_index)

In [ ]:
train_y_orgin

,Label
80630,BENIGN
62017,DDoS
5005,BENIGN
56849,DDoS
42434,BENIGN
...,...
50057,BENIGN
98047,DDoS
5192,BENIGN
77708,BENIGN


In [ ]:
df3= pd.concat([train_X_orgin, df2, train_y_orgin], axis=1)

In [ ]:
shap_values_test = explainer(test_X)

In [ ]:
df4 = pd.DataFrame(shap_values_test.values, columns=['shap_values'], index=test_index)

In [ ]:
df5 = pd.concat([test_X_orgin, df4, test_y_orgin], axis=1)

In [ ]:
df3

,Subflow Fwd Bytes,shap_values,Label
80630,24,1.705084,BENIGN
62017,24,1.705084,DDoS
5005,70,-2.693884,BENIGN
56849,56,1.562161,DDoS
42434,6,-2.738592,BENIGN
...,...,...,...
50057,66,-2.693884,BENIGN
98047,56,1.562161,DDoS
5192,611,-2.928717,BENIGN
77708,56,1.562161,BENIGN


In [ ]:
df5

,Subflow Fwd Bytes,shap_values,Label
43660,26,1.728764,DDoS
87278,6,-2.738592,BENIGN
14317,144,-2.693884,BENIGN
81932,30,1.841793,DDoS
95321,6,-2.738592,BENIGN
...,...,...,...
994,66,-2.693884,BENIGN
42287,1375,-2.751949,BENIGN
4967,60,-2.365482,BENIGN
47725,26,1.728764,DDoS


In [ ]:
df6 = pd.concat([df3, df5], axis=0)

In [ ]:
df6

,Subflow Fwd Bytes,shap_values,Label
80630,24,1.705084,BENIGN
62017,24,1.705084,DDoS
5005,70,-2.693884,BENIGN
56849,56,1.562161,DDoS
42434,6,-2.738592,BENIGN
...,...,...,...
994,66,-2.693884,BENIGN
42287,1375,-2.751949,BENIGN
4967,60,-2.365482,BENIGN
47725,26,1.728764,DDoS


In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
# train_set, test_set = train_test_split(df3, test_size=0.3, random_state=seed)
# train_X = train_set.drop('Label', axis=1)
# train_y = train_set[['Label']]
# test_X = test_set.drop('Label', axis=1)
# test_y = test_set[['Label']]

# Scaler = StandardScaler()
# train_X = Scaler.fit_transform(train_X)
# test_X = Scaler.transform(test_X)

# labelencoder = LabelEncoder()
# test_y = labelencoder.fit_transform(test_y)

In [ ]:
train_X = df6.drop('Label', axis=1)
train_y = df6[['Label']]
full_index = train_X.index.tolist()

Scaler = StandardScaler()
train_X = Scaler.fit_transform(train_X)

labelencoder = LabelEncoder()
test_y = labelencoder.fit_transform(train_y)

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().


In [ ]:
from sklearn.cluster import KMeans
model2 = KMeans(n_clusters=2)
model2.fit(train_X)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=2, n_init=10, n_jobs=None, precompute_distances='auto',
       random_state=None, tol=0.0001, verbose=0)

In [ ]:
y_pred=model2.predict(train_X)

In [ ]:
for i in range(len(y_pred)):
  if y_pred[i] == 0:
    y_pred[i] = 1
  elif y_pred[i] == 1:
    y_pred[i] =0

In [ ]:
labels_d = make_value2index(y_test_ada['Label'])

In [ ]:
display_metrics(test_y, y_pred, labels_d)


Accuracy: 0.90

Micro Precision: 0.90
Micro Recall: 0.90
Micro F1-score: 0.90

Macro Precision: 0.90
Macro Recall: 0.89
Macro F1-score: 0.89

Weighted Precision: 0.90
Weighted Recall: 0.90
Weighted F1-score: 0.90

Classification Report

              precision    recall  f1-score   support

      BENIGN       0.90      0.84      0.87     40957
        DDoS       0.89      0.94      0.91     59043

    accuracy                           0.90    100000
   macro avg       0.90      0.89      0.89    100000
weighted avg       0.90      0.90      0.90    100000



wrong label recognize rate

In [ ]:
test_y

array([0, 1, 0, ..., 0, 1, 0])

In [ ]:
result_df = pd.DataFrame({'origin':test_y, 'cluster_pred':y_pred}, index=full_index, columns=['origin', 'cluster_pred'])

In [ ]:
result_df

,origin,cluster_pred
80630,0,1
62017,1,1
5005,0,0
56849,1,1
42434,0,0
...,...,...
994,0,0
42287,0,0
4967,0,0
47725,1,1


In [ ]:
different_index = result_df[result_df['origin'] != result_df['cluster_pred']].index.tolist()

In [ ]:
count = 0
for i in different_index:
  if i in DDoS_index or i in Benign_index:
    count += 1

In [ ]:
recongnize_rate = count /(len(DDoS_index) + len(Benign_index))

In [ ]:
(len(DDoS_index) + len(Benign_index))

10000

In [ ]:
len(different_index)

10299

In [ ]:
recongnize_rate 

0.9421